# Das Projekt

#### Gruppenmitglieder: Tim Reimer, Markus Roos, Ronja Schwarz

#### Szenario:
Eine große Supermarktkette betreibt 300 Filialen in Deutschland. Als Grundlage für die Personalplanung muss für jede Filiale entschieden werden, wie viele MitarbeiterInnen an einem bestimmten Tag in der
Filiale sein müssen. Der Bedarf richtet sich dabei nach der erwarteten Anzahl von Kunden bzw. dem erwarteten Umsatz. Bisher wird der Bedarf individuell von den FilialleiterInnen ermittelt. In dieser Aufgabe soll anhand von historischen Umsatzzahlen, Filial-, Angebots- und Wettbewerberdaten ein Modell entwickelt werden, das für jede Filiale den erwarteten Umsatz möglichst gut vorhersagt und damit allen FilialleiterInnen eine passgenaue Personalplanung ermöglicht.

## Import der benötigten Bibliotheken

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

### Import CSV-Dateien

In [2]:
df_stores = pd.read_csv('./data/dmml1_stores.csv')
df_test = pd.read_csv('./data/dmml1_test.csv')
df_train = pd.read_csv('./data/dmml1_train.csv')
df_merge = pd.merge(df_train, df_stores)

## 2.1 Aufgaben
Für die Entwicklung des Modells muss der gesamte Machine Learning Workflow durchlaufen werden,
von der Datenvorverarbeitung und -analyse über das Modelltraining verschiedener Modelle bis zur Auswertung der Modellgüte.
Bearbeiten Sie dazu folgende Aufgaben:

### Aufgabe 1 (10 Punkte)
Untersuchen Sie die Daten. Beantworten Sie dabei folgende Fragen bzw. Aufgaben:

1. Wie viele Datenpunkte gibt es pro Store?

In [3]:
task_1_1 = df_train.groupby('Store ID')[['Store ID']].agg(Datenpunkte_Pro_Store=('Store ID', 'count'))
display(task_1_1)

,Datenpunkte_Pro_Store
Store ID,
1,850
2,850
3,850
4,850
5,850
...,...
296,850
297,850
298,850


In [4]:
#Naheliegende Frage: Wie viele unterschiedliche Anzahl an Datenpunkten gibt es mit welcher Anzahl?
task_1_1.groupby('Datenpunkte_Pro_Store').size()

Datenpunkte_Pro_Store
666     44
849      1
850    255
dtype: int64

2. Visualisieren Sie die Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und beschreiben Sie typische Muster, die Sie erkennen.

In [5]:
#Alle Store IDs
unique_store_ids = df_train['Store ID'].unique()

# Zufällig 5 Store IDs auswählen mit sample
random_store_ids = pd.Series(unique_store_ids).sample(n=3, random_state=1)
print(random_store_ids)

# DataFrame filtern, um nur die ausgewählten Store IDs zu behalten
selected_df = df_train[df_train['Store ID'].isin(random_store_ids)]
selected_df

#Lineplot für Verkaufzahlen der Stores
df_viz = selected_df.groupby(['Store ID', 'Date'], as_index=False)[['Sales']].sum().sort_values(by='Date')
px.line(df_viz, x='Date', y='Sales', color='Store ID')

189    190
123    124
185    186
dtype: int64


In [6]:
#Innerhalb von nur 2014
df_viz = selected_df[(df_train['Date'] >= '2014-01-01') & (df_train['Date'] <= '2014-12-31')].groupby(['Store ID', 'Date'], as_index=False)[['Sales']].sum().sort_values(by='Date')
px.line(df_viz, x='Date', y='Sales', color='Store ID')

C:\Users\Markus\AppData\Local\Temp\ipykernel_11272\3665853670.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



3. Beschreiben Sie einige prägnante Zusammenhänge, die Ihnen zwischen verschiedenen Features auffallen.

In [7]:
#(Vermeintlich) Offensichtlich: Wenn ein Store geschlossen ist, dann gibt es auch keinen Umsatz
df_train.loc[df_train['Open'].eq(0) & df_train['Sales'].ne(0)]

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday


In [8]:
#Aussage bestätigt.

In [9]:
#Gibt es auffälligkeiten zwischen Ladenmodellen(StoreType) und der Sortimentstufe(Assortment)?
# Gruppiere nach 'StoreType' und 'Assortment' und zähle die Vorkommen
assortment_counts = df_stores.groupby(['StoreType', 'Assortment']).size().reset_index(name='Count')
assortment_counts

  StoreType Assortment  Count
0         a          a     99
1         a          c     67
2         b          a      2
3         b          b      1
4         c          a     17
5         c          c     16
6         d          a     36
7         d          c     62


Beobachtung zwischen den Features:
Für die Features StoreType und Assortment fällt auf, dass es manche Kombinationen gar nicht gibt.
Beispiele: 
    Storetype a und Assortment b(extra)
    Storetype b und Assortment c(extended)

In [10]:
#Tieferer einblick in unterschiedliche Ladenmodelle:
df_merge.groupby(['StoreType'],as_index=False).agg(Durchschnittliche_Entfernung_der_Konurrenz = ('CompetitionDistance','mean'), Summe_Sales = ('Sales', 'sum'), Anzahl_Sales = ('Sales','count'), Durchschnittliche_Sales= ('Sales', 'mean'))

,StoreType,Durchschnittliche_Entfernung_der_Konurrenz,Summe_Sales,Anzahl_Sales,Durchschnittliche_Sales
0,a,4867.477878,812088836,138155,5878.099497
1,b,913.333333,23838239,2550,9348.329020
2,c,4035.737560,161122055,27130,5938.888868
3,d,6950.876269,445973452,79068,5640.378560


Hier lässt sich viel erkennen. 
Zum Beispiel:
Storetype b hat in der Regel Konkurenz in der nähe und die Anzahl der Sales sowie der Umsatz am geringesten. Aber durchschnittlich trotzdem am meisten Sales!

Folgende Frage: Liegt es an Werbeaktionen? Macht b keine Werbeaktion und dafür die anderen StoreTypes mehr?

In [11]:
#Erstmal gucken wie groß der Anteil der Werbeaktionen ist.
df_merge['Promo'].value_counts(normalize=True)

Promo
0    0.61951
1    0.38049
Name: proportion, dtype: float64

Es gibt wirklich viele (Fast 40%) Werbeaktionen. Machen bestimmte Storetypes die ganzen Werbaktionen?

In [12]:
# Gruppiere nach 'StoreType' und summiere die Werte in der 'Promo'-Spalte
promo_by_storetype = df_merge.groupby('StoreType')['Promo'].sum()
promo_by_storetype

StoreType
a    52600
b      972
c    10322
d    30050
Name: Promo, dtype: int64

Das sagt uns noch nicht viel, da die Anzahl der jeweiligen Storetypes noch relevant ist.

In [13]:
amount_store_types = df_merge['StoreType'].value_counts()
amount_store_types.sort_index(inplace=True)
amount_store_types

StoreType
a    138155
b      2550
c     27130
d     79068
Name: count, dtype: int64

In [14]:
ratio = promo_by_storetype / amount_store_types
ratio

StoreType
a    0.380732
b    0.381176
c    0.380464
d    0.380053
dtype: float64

Erstaunlicher Zufall, unabhängig vom Storetype immer ungefähr 38% aller verkaufstage promoaktion

In [15]:
#Lässt sich auf die schnelle noch was über die Entfernung erkennen bei der Distanz zum Konkurrenten?
px.scatter(df_stores, x='Store ID', y='CompetitionDistance', color='StoreType')

### Aufgabe 2 (10 Punkte)
Führen Sie geeignete Vorverarbeitungsschritte durch, z.B. Behandlung von Ausreißern und fehlenden
Werten, Skalierung der Features.

Durchgeführt Vorverarbeitungsschritte:
- dmml1_train.csv
  - ```dmml1_train['Date']``` zu Datum umwandeln, Format wurde beibehalten

In [16]:
df_train['Date'] = pd.to_datetime(df_train['Date'], format='%Y-%m-%d')

### Aufgabe 3 (15 Punkte)
Generieren Sie neue Features, die für die Vorhersage des Umsatzes aussagekräftig sein könnten.

### Aufgabe 4 (30 Punkte)
Trainieren Sie drei verschiedene Modelle, die in der Vorlesung behandelt wurden: ein lineares Modell (einfache lineare Regression, Ridge, Lasso), einen Entscheidungsbaum und ein Ensemble-Modell (Gradient Boosting oder Random Forest)

1. Optimieren sie Hyperparameter der Modelle mittels Suche und Kreuzvalidierung. Überlegen Sie dazu zunächst (mit Hilfe der Vorlesungsunterlagen und der Dokumentation der Methoden in scikit-learn), was für die jeweiligen Modelle Hyperparameter sind und für welche sich eine Optimierung ggf. lohnen könnte.

2. Welches sind die wichtigsten Features für die jeweiligen Modelle?

### Aufgabe 5 (15 Punkte)
Das Ziel der Modelle ist es, die FilialleiterInnen bei der Erstellung der Schichtpläne zu unterstützen.
Dazu leiten diese aus der Umsatzvorhersage ihres Modells einen Bedarf an MitarbeiterInnen ab, die an
diesem Tag zur Schicht eingeteilt werden. Wenn die Umsatzvorhersage deutlich höher ist als der tatsächliche Umsatz, so werden zu viele Mitarbeiter in der Filiale sein und es entstehen unnötige Lohnkosten. Nehmen Sie an, dass pro Tag, an dem die Vorhersage den tatsächlichen Umsatz um mehr als 3000
EUR überschätzt 150 EUR Kosten (zusätzliche Lohnkosten) anfallen. Wenn die Vorhersage um mehr
als 6000 EUR höher liegt, betragen die Kosten sogar 250 EUR. Falls die Vorhersage den tatsächlichen
Umsatz unterschätzt, so sind die Kosten schwieriger zu quantifizieren. Darunter fallen Kosten für etwaige Überstunden aber auch Kosten durch Unzufriedenheit der Kunden und der Mitarbeiter (da z.B.
lange Wartezeiten in Kauf genommen werden müssen), die sich langfristig negativ auswirken. Nehmen
Sie vereinfachend an, dass 100 EUR Kosten entstehen, wenn das Modell den Umsatz um mehr als 4000
EUR unterschätzt. Welches ihrer Modelle wird nach diesen Annahmen voraussichtlich die geringsten
Gesamtkosten verursachen?

### Aufgabe 6 (20 Punkte)
Erstellen Sie eine Vorhersage für die Testdaten mit dem besten Modell (bezüglich ihrer Metrik aus der
vorherigen Aufgabe). Geben Sie diese Vorhersage als .csv Datei mit ab. Fügen Sie dazu einfach eine
Spalte Sales zur Datei dmml1_test.csv hinzu.